# **Enervia chat-bot**

- Extraer y Segmentar: De los PDFs, extrae y segmenta el texto.
- Generar Embeddings: Convierte cada fragmento en un vector.
- Indexar: Almacena estos vectores en una base de datos vectorial (ej. FAISS).
- Consulta y Recuperación: Cuando el usuario consulta, busca los fragmentos relevantes.
- Construir Prompt y Responder: Integra los fragmentos en un prompt y genera la respuesta con ChatGPT.


## Parte 1: Extraer y Segmentar
### De los PDFs, extrae y segmenta el texto


In [1]:
import PyPDF2

def extraer_texto_pdf(ruta_pdf):
    """
    Extrae y retorna el texto completo del PDF ubicado en 'ruta_pdf'.
    - Abre el archivo en modo binario.
    - Utiliza PyPDF2.PdfReader para leer cada página.
    - Extrae y concatena el texto de cada página.
    - Permite verificar el contenido extraído imprimiendo un fragmento.
    """
    texto_completo = ""
    with open(ruta_pdf, "rb") as archivo:
        lector = PyPDF2.PdfReader(archivo)
        for pagina in lector.pages:
            texto = pagina.extract_text()
            if texto:  # Verificar que se haya extraído texto
                texto_completo += texto + "\n"
    return texto_completo

# Ejemplo de uso:
ruta_dossier = "contenido_para_alimentar/guia_enervia.pdf"  # PDF del dossier unificado
ruta_faq = "contenido_para_alimentar/ENERVIA_faqs.pdf"       # PDF de FAQs

texto_dossier = extraer_texto_pdf(ruta_dossier)
texto_faq = extraer_texto_pdf(ruta_faq)

print("Extracto del dossier:")
print(texto_dossier[:500])  # Muestra los primeros 500 caracteres

print("\nExtracto del PDF de FAQs:")
print(texto_faq[:500])


Extracto del dossier:
guia_enervia.md 2025-03-10
1 / 91. Introducción
Resumen Objetivo
Este documento es un dossier integral que consolida toda la información relevante sobre Enervia, recopilada a
partir de diversas fuentes oficiales y externas. Su finalidad es ofrecer una visión completa y estructurada de la
empresa, abarcando desde su historia y misión hasta la descripción detallada de sus servicios, opiniones de
clientes y presencia digital. Esta recopilación servirá como base para posteriores procesos, como la ge

Extracto del PDF de FAQs:
ENER VIA_faqs.md 2025-03-10
1 / 2ENERVIA – F AQ (Pr eguntas Fr ecuent es)
1. ¿Qué es Ener via y a qué se dedica?
Enervia es una empresa especializada en soluciones ener géticas  , enfocada principalmente en la instalación,
mant enimient o y r eparación de SAIs (UPS) . Además, ofrecemos servicios complementarios de ener gía
solar  para quienes buscan optimizar su consumo y apostar por fuentes renovables.
2. ¿P or qué elegir Ener via p ara mis nece

### Segmentación del contenido.

Una vez extraído el texto, es recomendable segmentarlo en secciones lógicas (por ejemplo, separando por secciones, párrafos o títulos). Esto facilitará la generación de embeddings más adelante. Algunas ideas:

In [2]:
import re

def segmentar_por_encabezados(textos):
    """
    Divide el texto en fragmentos utilizando encabezados numéricos (Ej: "1. Introducción") como delimitadores.
    Retorna una lista de secciones grandes.
    """
    secciones = []
    for texto in textos:
        # Patrón para encabezados tipo "1." "2." etc. (al inicio de línea, opcional salto de línea)
        # Conservamos el encabezado en el siguiente fragmento usando lookahead.
        pattern = r'(?=\n?\d+\.\s)'
        secciones_texto = re.split(pattern, texto)
        secciones_texto = [sec.strip() for sec in secciones_texto if sec.strip()]
        secciones.append(secciones_texto)
    return secciones

def segmentar_por_subtitulos_o_parrafos(texto):
    """
    Segmenta por saltos de línea dobles o subtítulos típicos (por ejemplo, 'Resumen Objetivo', 'Objetivos', etc.).
    Puedes ajustar el patrón para reconocer más casos.
    """
    # Dividimos por doble salto de línea o salto de línea que contenga un título aparente.
    # El patrón se puede ajustar según la estructura real de tu documento.
    pattern = r'(\n\s*\n)'  # doble salto de línea
    subfragmentos = re.split(pattern, texto)
    
    # Recompone los subfragmentos sin perder la información del separador, 
    # pero en este caso, con una división "fuerte" en párrafos.
    # Filtramos fragmentos vacíos y limpiamos espacios.
    subfragmentos = [frag.strip() for frag in subfragmentos if frag.strip() and frag != pattern]
    return subfragmentos

def dividir_por_longitud(fragmentos, max_palabras=300):
    """
    Si un fragmento excede max_palabras, lo dividimos en trozos de max_palabras cada uno.
    Retorna una nueva lista de fragmentos con la longitud adecuada.
    """
    resultado = []
    for frag in fragmentos:
        palabras = frag.split()
        if len(palabras) <= max_palabras:
            resultado.append(frag)
        else:
            # dividimos en trozos de max_palabras
            for i in range(0, len(palabras), max_palabras):
                chunk = " ".join(palabras[i:i+max_palabras])
                resultado.append(chunk)
    return resultado

def segmentar_documento(textos, max_palabras=300):
    # 1) Dividir por encabezados principales (1., 2., 3., etc.) para cada documento
    secciones_principales = segmentar_por_encabezados(textos)
    
    # 2) Aplanar las secciones y para cada sección, dividir por subtítulos o párrafos
    subfragmentos_totales = []
    for secciones_doc in secciones_principales:  # Cada documento (lista de secciones)
        for seccion in secciones_doc:             # Cada sección individual (string)
            subfragmentos = segmentar_por_subtitulos_o_parrafos(seccion)
            subfragmentos_totales.extend(subfragmentos)
    
    # 3) Dividir los subfragmentos largos por longitud
    fragmentos_finales = dividir_por_longitud(subfragmentos_totales, max_palabras)
    return fragmentos_finales


# EJEMPLO DE USO:
if __name__ == "__main__":
    # texto_extraido sería el contenido completo del PDF que obtuviste con PyPDF2
    textos_extraidos = [texto_dossier, texto_faq]

    fragmentos = segmentar_documento(textos_extraidos, max_palabras=300)
    
    # Ver resultados
    print(f"Total de fragmentos: {len(fragmentos)}")
    for i, frag in enumerate(fragmentos, start=1):
        num_palabras = len(frag.split())
        print(f"Fragmento {i} ({num_palabras} palabras):\n{frag}\n{'-'*80}")


Total de fragmentos: 37
Fragmento 1 (4 palabras):
guia_enervia.md 2025-03-10
1 /
--------------------------------------------------------------------------------
Fragmento 2 (1 palabras):
9
--------------------------------------------------------------------------------
Fragmento 3 (300 palabras):
1. Introducción Resumen Objetivo Este documento es un dossier integral que consolida toda la información relevante sobre Enervia, recopilada a partir de diversas fuentes oficiales y externas. Su finalidad es ofrecer una visión completa y estructurada de la empresa, abarcando desde su historia y misión hasta la descripción detallada de sus servicios, opiniones de clientes y presencia digital. Esta recopilación servirá como base para posteriores procesos, como la generación de embeddings y la implementación de un chatbot que brinde respuestas precisas y contextuales a consultas de usuarios. Objetiv os Centralizar la Infor mación: Unificar y estructurar la información proveniente de la web ofici

In [3]:
import re
def clean_text(text):
    """
    Limpia el texto aplicando correcciones específicas y normalizando espacios.
    """
    # Diccionario con errores comunes y sus correcciones
    corrections = {
        "Infor mación": "Información",
        "Sopor te": "Soporte",
        "Compr omiso": "Compromiso",
        "Pr oyectos": "Proyectos",
        "Cont enido": "Contenido",
        "T estimonios": "Testimonios",
        "Client e": "Cliente",
        "W eb": "Web",
        "Mant enimient o": "Mantenimiento",
        "T écnicas": "Técnicas",
        "Ener gías R enov ables": "Energías Renovables",
        "Pr esentaciones Int ernas": "Presentaciones Internas",
        "R euniones": "Reuniones",
        "Mark eting": "Marketing",
        "Objetiv os": "Objetivos"
    }
    
    # Aplicar las correcciones definidas
    for error, correct in corrections.items():
        text = text.replace(error, correct)
    
    # Normalizar espacios: reemplazar saltos de línea y múltiples espacios por un solo espacio
    text = re.sub(r'\s+', ' ', text)
    return text.strip()



# fragmentos[9] = fragmentos[9] + " " + fragmentos[10]

# fragmentos[12] = fragmentos[11] + " " + fragmentos[12]

# del(fragmentos[0], fragmentos[0], fragmentos[9])

# Suponiendo que 'fragmentos_dossier' es la lista de fragmentos extraídos
clean_fragments = [clean_text(fragment) for fragment in fragmentos]

print(f"Total de fragmentos: {len(clean_fragments)}")
for i, frag in enumerate(clean_fragments, start=1):
    num_palabras = len(frag.split())
    print(f"Fragmento {i} ({num_palabras} palabras):\n{frag}\n{'-'*80}")


Total de fragmentos: 37
Fragmento 1 (4 palabras):
guia_enervia.md 2025-03-10 1 /
--------------------------------------------------------------------------------
Fragmento 2 (1 palabras):
9
--------------------------------------------------------------------------------
Fragmento 3 (293 palabras):
1. Introducción Resumen Objetivo Este documento es un dossier integral que consolida toda la información relevante sobre Enervia, recopilada a partir de diversas fuentes oficiales y externas. Su finalidad es ofrecer una visión completa y estructurada de la empresa, abarcando desde su historia y misión hasta la descripción detallada de sus servicios, opiniones de clientes y presencia digital. Esta recopilación servirá como base para posteriores procesos, como la generación de embeddings y la implementación de un chatbot que brinde respuestas precisas y contextuales a consultas de usuarios. Objetivos Centralizar la Información: Unificar y estructurar la información proveniente de la web oficial

In [4]:
# Calcular la longitud de cada fragmento
longitudes = [len(fragment.split()) for fragment in clean_fragments]

# Calcular la longitud media
longitud_media = sum(longitudes) / len(clean_fragments)

print("Longitud media de los fragmentos:", longitud_media)

Longitud media de los fragmentos: 95.56756756756756


## Parte 2: Generar embeddings
### Convierte cada fragmento en un vector

In [5]:
# Instala la biblioteca si no la tienes instalada:
# pip install sentence-transformers

from sentence_transformers import SentenceTransformer

# Cargar el modelo preentrenado (puedes cambiarlo según tus necesidades)
modelo = SentenceTransformer('all-MiniLM-L6-v2')

# Generar embeddings para cada fragmento limpio
embeddings_gratis = modelo.encode(clean_fragments, convert_to_numpy=True)

# Verificar la dimensión del primer embedding
print("Dimensión del primer embedding:", embeddings_gratis[0].shape)



Dimensión del primer embedding: (384,)


## Parte 3: Indexar
### Almacena estos vectores en una base de datos vectorial (ej. FAISS)

Indexar los embeddings en FAISS

In [6]:
import faiss
import numpy as np

# Convierte la lista de embeddings a un array de NumPy (si no lo está ya)
emb_array = np.array(embeddings_gratis, dtype='float32')  # Asegúrate de que sea float32

# Dimensión de cada embedding
dimension = emb_array.shape[1]

# Creamos un índice FAISS plano que use L2 (distancia euclidiana)
index = faiss.IndexFlatL2(dimension)

# Añadimos los embeddings al índice
index.add(emb_array)

print(f"Total de vectores indexados: {index.ntotal}")

Total de vectores indexados: 37


Buscar los fragmentos más similares

In [7]:
def buscar_fragmentos(query, k=3):
    """
    Retorna los índices de los k fragmentos más similares a la consulta.
    """
    # Generar embedding de la consulta
    query_embedding = modelo.encode([query], convert_to_numpy=True).astype('float32')
    
    # Realizar la búsqueda en FAISS
    # D: Distancias, I: Índices de los vectores
    D, I = index.search(query_embedding, k)
    return I[0], D[0]

# Ejemplo de uso:
consulta = "¿Qué servicios de mantenimiento ofrece Enervía?"
indices_similares, distancias = buscar_fragmentos(consulta, k=3)

for idx, dist in zip(indices_similares, distancias):
    print(f"Índice: {idx}, Distancia: {dist}")
    print(f"Fragmento:\n{clean_fragments[idx]}")
    print("-"*80)

Índice: 17, Distancia: 0.7450803518295288
Fragmento:
1. ¿Qué es Ener via y a qué se dedica? Enervia es una empresa especializada en soluciones ener géticas , enfocada principalmente en la instalación, mant enimient o y r eparación de SAIs (UPS) . Además, ofrecemos servicios complementarios de ener gía solar para quienes buscan optimizar su consumo y apostar por fuentes renovables.
--------------------------------------------------------------------------------
Índice: 18, Distancia: 0.7801899909973145
Fragmento:
2. ¿P or qué elegir Ener via p ara mis necesidades de ener gía? - Exper iencia : contamos con años de trayectoria en el sector de SAIs y energías renovables. - Equipo t écnico cer tificado : nuestros profesionales están capacitados para resolver cualquier incidencia. - Atención per sonalizada : adaptamos cada proyecto a las necesidades concretas de cada cliente. - Soporte int egral : cubrimos instalación, mantenimiento preventivo/correctivo y servicio de asistencia.
-----------

In [ ]:
import openai

openai.api_key = "API-KEY"

def generar_respuesta_con_contexto(query, fragmentos_relevantes):
    """
    Envía un prompt a la nueva API de ChatCompletion usando GPT-3.5-turbo.
    """
    # Construye un mensaje "system" con la instrucción principal
    system_message=[
    {"role": "system", "content": 
            "Eres un asistente virtual de Enervía. Usa la siguiente información para responder de forma "
            "clara y concisa a la pregunta del usuario. No inventes datos si no aparecen en los fragmentos.\n\n"
            f"Información relevante:\n{fragmentos_relevantes}"
            }
    ]

    user_message=[
    {"role": "user", "content": query}]    

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=system_message + user_message,  # Concatenamos las listas
        temperature=0.7,
        max_tokens=500
    )


    # La respuesta se encuentra en response["choices"][0]["message"]["content"]
    return response["choices"][0]["message"]["content"].strip()

# Ejemplo de uso
def buscar_fragmentos(query, k=3):
    # Aquí asumimos que ya tienes tu index FAISS y el modelo para embeddings
    query_embedding = modelo.encode([query], convert_to_numpy=True).astype('float32')
    D, I = index.search(query_embedding, k)
    return I[0], D[0]

consulta = "¿Con qué tipos de clientes trabaja Enervia?"
indices_similares, distancias = buscar_fragmentos(consulta, k=3)

# Une los fragmentos relevantes en un solo string
fragmentos_relevantes = "\n".join(clean_fragments[idx] for idx in indices_similares)

respuesta = generar_respuesta_con_contexto(consulta, fragmentos_relevantes)
print("Chatbot:", respuesta)

Chatbot: Enervía trabaja con empresas de cualquier sector, organismos públicos, industrias, pymes e incluso particulares que requieran asegurar continuidad eléctrica en su vivienda o negocio. La empresa se adapta al tamaño y la complejidad de cada proyecto.
